In [21]:
# %% [markdown]
# # LoRA Finetuning: Qwen2.5-0.5B on Itihasa Dataset
# Optimized for Apple Silicon (M4 Mac) - Working Version

# %% [markdown]
# ## 1. Setup and Imports

# %%
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model, TaskType

# Check for MPS availability
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# %% [markdown]
# ## 2. Configuration

# %%
# Model configuration
MODEL_NAME = "Qwen/Qwen2.5-0.5B"
OUTPUT_DIR = "./qwen-itihasa-lora"

# Dataset subset configuration
TRAIN_SUBSET_SIZE = 20000  # Use first 1000 samples for training
VAL_SUBSET_SIZE = 500     # Use first 200 samples for validation

# Training configuration
MAX_SEQ_LENGTH = 256
NUM_EPOCHS = 1
BATCH_SIZE = 8  # Reduced for stability
GRADIENT_ACCUM_STEPS = 4
LEARNING_RATE = 1e-5

print(f"Training subset: {TRAIN_SUBSET_SIZE} samples")
print(f"Validation subset: {VAL_SUBSET_SIZE} samples")
print(f"Max sequence length: {MAX_SEQ_LENGTH}")
print(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUM_STEPS}")

# %% [markdown]
# ## 3. Load Dataset from Local Parquet Files

# %%
# Load dataset from local parquet files
print("Loading Itihasa dataset from local parquet files...")
dataset = load_dataset(
    "parquet",
    data_files={
        "train": "ithasa/train/0000.parquet",
        "validation": "ithasa/val/0000.parquet",
        "test": "ithasa/test/0000.parquet"
    }
)

print(f"\nDataset structure:")
print(f"Train samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")
print(f"Test samples: {len(dataset['test'])}")

# %% [markdown]
# ## 4. Explore Dataset

# %%
# Show sample data
print("\n" + "="*80)
print("Sample from dataset:")
print("="*80)
sample = dataset['train'][0]
print(f"\nFull sample structure: {sample.keys()}")
print(f"\nSample data:")
print(sample)

# %% [markdown]
# ## 5. Create Dataset Subsets

# %%
# Create subsets
print(f"\nCreating subsets...")
train_subset = dataset["train"].select(range(min(TRAIN_SUBSET_SIZE, len(dataset["train"]))))
val_subset = dataset["validation"].select(range(min(VAL_SUBSET_SIZE, len(dataset["validation"]))))

print(f"Train subset: {len(train_subset)} samples")
print(f"Validation subset: {len(val_subset)} samples")

# %% [markdown]
# ## 6. Load Tokenizer and Model

# %%
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Tokenizer loaded. Vocab size: {len(tokenizer)}")

# %%
# Load base model
print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to(device)

print(f"Model loaded. Parameters: {model.num_parameters():,}")

# %% [markdown]
# ## 7. Configure and Apply LoRA

# %%
# LoRA configuration
print("Configuring LoRA...")
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# %% [markdown]
# ## 8. Preprocessing Function

# %%
def tokenize_fn(examples):
    """
    Tokenize with proper prompt masking.
    Only train on the output (English translation), not the input prompt (Sanskrit).
    """
    all_input_ids = []
    all_attention_masks = []
    all_labels = []
    
    for translation in examples['translation']:
        # Extract Sanskrit and English
        sn_text = str(translation['sn']).strip()
        en_text = str(translation['en']).strip()
        
        # Create prompt
        prompt = f"Translate this Sanskrit shloka to English:\n{sn_text}\n\nTranslation:"
        
        # Tokenize prompt separately to get its length
        prompt_tokens = tokenizer(prompt, add_special_tokens=True)
        prompt_len = len(prompt_tokens["input_ids"])
        
        # Create full sequence: prompt + output + eos
        full_text = prompt + " " + en_text + tokenizer.eos_token
        
        # Tokenize full sequence
        encoding = tokenizer(
            full_text,
            max_length=MAX_SEQ_LENGTH,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        
        input_ids = encoding["input_ids"][0].tolist()
        attention_mask = encoding["attention_mask"][0].tolist()
        
        # Create labels: mask prompt, keep output
        labels = input_ids.copy()
        
        # Mask the prompt tokens
        for i in range(min(prompt_len, len(labels))):
            labels[i] = -100
        
        # Mask padding tokens
        for i in range(len(labels)):
            if attention_mask[i] == 0:
                labels[i] = -100
        
        all_input_ids.append(input_ids)
        all_attention_masks.append(attention_mask)
        all_labels.append(labels)
    
    return {
        "input_ids": all_input_ids,
        "attention_mask": all_attention_masks,
        "labels": all_labels
    }

# %% [markdown]
# ## 9. Tokenize Datasets

# %%
# Tokenize datasets
print("Tokenizing datasets...")
tokenized_train = train_subset.map(
    tokenize_fn,
    batched=True,
    batch_size=100,
    remove_columns=train_subset.column_names,
    desc="Tokenizing training data"
)

tokenized_val = val_subset.map(
    tokenize_fn,
    batched=True,
    batch_size=100,
    remove_columns=val_subset.column_names,
    desc="Tokenizing validation data"
)

print(f"✅ Tokenization complete")
print(f"Tokenized train samples: {len(tokenized_train)}")
print(f"Tokenized val samples: {len(tokenized_val)}")

# %% [markdown]
# ## 10. Debug Tokenization (Optional)

# %%
# Check a few examples
print("\n" + "="*80)
print("DEBUGGING TOKENIZED DATA")
print("="*80)

for i in range(min(3, len(tokenized_train))):
    example = tokenized_train[i]
    
    print(f"\n--- Example {i} ---")
    
    non_masked = sum(1 for label in example['labels'] if label != -100)
    masked = sum(1 for label in example['labels'] if label == -100)
    padding = sum(1 for mask in example['attention_mask'] if mask == 0)
    actual_tokens = len(example['input_ids']) - padding
    
    print(f"Sequence length: {len(example['input_ids'])}")
    print(f"Actual tokens (no padding): {actual_tokens}")
    print(f"Padding tokens: {padding}")
    print(f"Non-masked tokens (trainable): {non_masked}")
    print(f"Masked tokens: {masked}")
    print(f"Percentage trainable: {non_masked / len(example['labels']) * 100:.1f}%")
    
    if non_masked == 0:
        print("⚠️ WARNING: No trainable tokens!")

print("="*80)

# %% [markdown]
# ## 11. Setup Training

# %%
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUM_STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=50,
    eval_steps=50,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    warmup_steps=25,
    fp16=True,
    save_total_limit=3,
    dataloader_num_workers=0,
    remove_unused_columns=False,
    report_to="none",
    push_to_hub=False,
    max_grad_norm=1.0,
)

print("📝 Training Configuration:")
print(f"  - Max sequence length: {MAX_SEQ_LENGTH}")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Gradient accumulation: {GRADIENT_ACCUM_STEPS}")
print(f"  - Effective batch size: {BATCH_SIZE * GRADIENT_ACCUM_STEPS}")
print(f"  - Learning rate: {LEARNING_RATE}")
print(f"  - Total epochs: {NUM_EPOCHS}")

# %%
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

print("Trainer initialized successfully!")

# %% [markdown]
# ## 12. Train the Model

# %%
# Start training
print("\n" + "="*80)
print("🚀 Starting training...")
print("="*80 + "\n")

trainer.train()

print("\n" + "="*80)
print("✅ Training complete!")
print("="*80)

# %% [markdown]
# ## 13. Save Model

# %%
# Save the final model
print(f"\nSaving model to {OUTPUT_DIR}...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Model saved successfully!")
print(f"✓ Location: {OUTPUT_DIR}")

# %% [markdown]
# ## 14. Quick Test

# %%
# Test the model with a sample
print("\n" + "="*80)
print("Testing the trained model...")
print("="*80)

test_sanskrit = "धर्मो रक्षति रक्षितः"
test_prompt = f"Translate this Sanskrit shloka to English:\n{test_sanskrit}\n\nTranslation:"

inputs = tokenizer(test_prompt, return_tensors="pt").to(device)
outputs = model.generate(
    **inputs,
    max_length=200,
    num_return_sequences=1,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\n📜 Sanskrit: {test_sanskrit}")
print(f"\n🔄 Generated output:\n{generated_text}")
print("\n" + "="*80)

# %% [markdown]
# ## Done!
# Your LoRA adapter is now trained and saved. Load it later using:
# ```python
# from peft import PeftModel
# base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B")
# model = PeftModel.from_pretrained(base_model, "./qwen-itihasa-lora")
# ```

Using device: mps
Training subset: 20000 samples
Validation subset: 500 samples
Max sequence length: 256
Effective batch size: 32
Loading Itihasa dataset from local parquet files...

Dataset structure:
Train samples: 75162
Validation samples: 6149
Test samples: 11722

Sample from dataset:

Full sample structure: dict_keys(['translation'])

Sample data:
{'translation': {'en': 'The ascetic Vālmīki asked Nārada, the best of sages and foremost of those conversant with words, ever engaged in austerities and Vedic studies.', 'sn': 'ॐ तपः स्वाध्यायनिरतं तपस्वी वाग्विदां वरम्। नारदं परिपप्रच्छ वाल्मीकिर्मुनिपुङ्गवम्॥'}}

Creating subsets...
Train subset: 20000 samples
Validation subset: 500 samples
Loading tokenizer...
Tokenizer loaded. Vocab size: 151665
Loading base model...
Model loaded. Parameters: 494,032,768
Configuring LoRA...
trainable params: 540,672 || all params: 494,573,440 || trainable%: 0.1093
Tokenizing datasets...


Tokenizing training data:   0%|          | 0/20000 [00:00<?, ? examples/s]

✅ Tokenization complete
Tokenized train samples: 20000
Tokenized val samples: 500

DEBUGGING TOKENIZED DATA

--- Example 0 ---
Sequence length: 256
Actual tokens (no padding): 136
Padding tokens: 120
Non-masked tokens (trainable): 40
Masked tokens: 216
Percentage trainable: 15.6%

--- Example 1 ---
Sequence length: 256
Actual tokens (no padding): 134
Padding tokens: 122
Non-masked tokens (trainable): 31
Masked tokens: 225
Percentage trainable: 12.1%

--- Example 2 ---
Sequence length: 256
Actual tokens (no padding): 129
Padding tokens: 127
Non-masked tokens (trainable): 35
Masked tokens: 221
Percentage trainable: 13.7%
📝 Training Configuration:
  - Max sequence length: 256
  - Batch size: 8
  - Gradient accumulation: 4
  - Effective batch size: 32
  - Learning rate: 1e-05
  - Total epochs: 1
Trainer initialized successfully!

🚀 Starting training...



Step,Training Loss,Validation Loss
50,4.054500,4.043245
100,3.986000,3.963691
150,3.867300,3.895544
200,3.870100,3.843967
250,3.812100,3.809147
300,3.785700,3.784872
350,3.765400,3.767917
400,3.739200,3.756110
450,3.739700,3.747189
500,3.748800,3.741186


/Users/shreekrishnajamakatel/personal/llm/llm_model_test/qwen-env/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/shreekrishnajamakatel/personal/llm/llm_model_test/qwen-env/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/shreekrishnajamakatel/personal/llm/llm_model_test/qwen-env/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/shreekrishnajamakatel/personal/llm/llm_model_test/qwen-env/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true 


✅ Training complete!

Saving model to ./qwen-itihasa-lora...


Both `max_new_tokens` (=2048) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


✅ Model saved successfully!
✓ Location: ./qwen-itihasa-lora

Testing the trained model...

📜 Sanskrit: धर्मो रक्षति रक्षितः

🔄 Generated output:
Translate this Sanskrit shloka to English:
धर्मो रक्षति रक्षितः

Translation: The ruler was overcome by fear and was unable to speak.



In [27]:
# %%
# Testing Trained Qwen-Itihasa LoRA Model
# Test on multiple Sanskrit samples from Mahabharata & Ramayana

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Device
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# %%
# Load the Trained Model
# Paths
BASE_MODEL = "Qwen/Qwen2.5-0.5B"
LORA_MODEL = "./qwen-itihasa-lora"

print("Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    trust_remote_code=True
).to(device)

print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(base_model, LORA_MODEL)
model.eval()  # Set to evaluation mode

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(LORA_MODEL)

print("✅ Model loaded successfully!\n")

# %%
# Test samples from Mahabharata and Ramayana

# Sample Sanskrit texts for testing - Mahabharata & Ramayana
test_samples = [
    # MAHABHARATA
    {
        "text": "धृतराष्ट्र उवाच। धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सवः। मामकाः पाण्डवाश्चैव किमकुर्वत सञ्जय॥",
        "source": "Mahabharata - Bhagavad Gita 1.1",
        "context": "Dhritarashtra's opening question about the battle"
    },
    {
        "text": "यदा यदा हि धर्मस्य ग्लानिर्भवति भारत। अभ्युत्थानमधर्मस्य तदात्मानं सृजाम्यहम्॥",
        "source": "Mahabharata - Bhagavad Gita 4.7",
        "context": "Krishna on divine incarnation"
    },
    {
        "text": "कर्मण्येवाधिकारस्ते मा फलेषु कदाचन। मा कर्मफलहेतुर्भूर्मा ते सङ्गोऽस्त्वकर्मणि॥",
        "source": "Mahabharata - Bhagavad Gita 2.47",
        "context": "Famous verse on karma yoga"
    },
    {
        "text": "सत्यं ब्रूयात् प्रियं ब्रूयात् न ब्रूयात् सत्यमप्रियम्। प्रियं च नानृतं ब्रूयात् एष धर्मः सनातनः॥",
        "source": "Mahabharata - Udyoga Parva",
        "context": "Principles of truthful speech"
    },
    {
        "text": "अहिंसा परमो धर्मः धर्म हिंसा तथैव च। तस्मात् धर्मः परो राजन् सर्वेषामिव वै मतः॥",
        "source": "Mahabharata - Shanti Parva",
        "context": "On non-violence and righteous violence"
    },
    
    # RAMAYANA
    {
        "text": "विश्वामित्रवचः श्रुत्वा राघवः सहलक्ष्मणः। विस्मयं परमं गत्वा विश्वामित्रमथाब्रवीत्॥",
        "source": "Ramayana - Bala Kanda",
        "context": "Rama and Lakshmana listening to Vishwamitra"
    },
    {
        "text": "रामो विग्रहवान् धर्मः साधुः सत्यपराक्रमः। राजा सर्वस्य लोकस्य देवानामिव वासवः॥",
        "source": "Ramayana - Ayodhya Kanda",
        "context": "Description of Rama's virtues"
    },
    {
        "text": "न जातु कामान्न भयान्न लोभाद् धर्मं त्यजेयं प्राणैरपि गुरीयसम्। न जीवितस्याकाङ्क्षा मे न च राज्यस्य कर्हिचित्॥",
        "source": "Ramayana - Ayodhya Kanda",
        "context": "Rama's commitment to dharma"
    },
    {
        "text": "मातृदेवो भव पितृदेवो भव आचार्यदेवो भव अतिथिदेवो भव।",
        "source": "Ramayana",
        "context": "Respect for elders and guests"
    },
    {
        "text": "सीता मम प्राणेभ्योऽपि गरीयसी। जानकीमनृतामकृत्वा न शक्तः सुखमाप्तुम्॥",
        "source": "Ramayana - Aranya Kanda",
        "context": "Rama's love for Sita"
    }
]

print(f"Loaded {len(test_samples)} test samples")

# %%
# Translation Function
def translate_sanskrit(text, max_new_tokens=128, temperature=0.7, num_beams=1):
    """
    Translate Sanskrit text to English using the trained model.
    
    Args:
        text: Sanskrit text to translate
        max_new_tokens: Maximum number of new tokens to generate
        temperature: Sampling temperature (lower = more conservative)
        num_beams: Number of beams for beam search (1 = greedy/sampling)
    """
    # Create prompt in the same format as training
    prompt = f"Translate this Sanskrit shloka to English:\n{text}\n\nTranslation:"
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,  # Changed from max_length
            num_return_sequences=1,
            temperature=temperature,
            do_sample=True if num_beams == 1 else False,
            num_beams=num_beams,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # Decode
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the translation (remove the prompt)
    if "Translation:" in generated_text:
        translation = generated_text.split("Translation:")[-1].strip()
    else:
        translation = generated_text
    
    return translation

print("✅ Translation function ready")

# %%
# Run Tests on All Samples
print("="*100)
print("SANSKRIT TO ENGLISH TRANSLATION TESTS")
print("="*100)

results = []

for i, sample in enumerate(test_samples, 1):
    print(f"\n{'─'*100}")
    print(f"Test {i}/{len(test_samples)}: {sample['source']}")
    print(f"{'─'*100}")
    
    print(f"\n📜 Sanskrit:")
    print(f"   {sample['text']}")
    
    print(f"\n🎯 Expected Context:")
    print(f"   {sample['context']}")
    
    print(f"\n🔄 Translating...")
    translation = translate_sanskrit(sample['text'])
    
    print(f"\n✨ Model Translation:")
    print(f"   {translation}")
    
    # Store results
    results.append({
        "source": sample['source'],
        "sanskrit": sample['text'],
        "context": sample['context'],
        "generated": translation
    })
    
    print()

print("="*100)
print("✅ All translations completed!")
print("="*100)

# %%
# Summary Results
print("\n" + "="*100)
print("SUMMARY OF TRANSLATIONS")
print("="*100)

for i, result in enumerate(results, 1):
    print(f"\n{i}. {result['source']}")
    print(f"   Sanskrit: {result['sanskrit'][:60]}...")
    print(f"   Generated: {result['generated'][:100]}...")
    print()

# %%
# Test with Different Generation Strategies
print("\n" + "="*100)
print("TESTING DIFFERENT GENERATION STRATEGIES")
print("="*100)

test_text = "धर्मो रक्षति रक्षितः"

strategies = [
    {"name": "Low Temperature (Conservative)", "temperature": 0.3, "num_beams": 1},
    {"name": "Medium Temperature (Balanced)", "temperature": 0.7, "num_beams": 1},
    {"name": "High Temperature (Creative)", "temperature": 1.0, "num_beams": 1},
    {"name": "Beam Search (Deterministic)", "temperature": 0.7, "num_beams": 3},
]

print(f"\n📜 Test Sanskrit: {test_text}")
print(f"🎯 Expected: Dharma protects those who protect it\n")

for strategy in strategies:
    print(f"\n{'─'*100}")
    print(f"Strategy: {strategy['name']}")
    print(f"  Temperature: {strategy['temperature']}, Beams: {strategy['num_beams']}")
    print(f"{'─'*100}")
    
    translation = translate_sanskrit(
        test_text,
        temperature=strategy['temperature'],
        num_beams=strategy['num_beams']
    )
    
    print(f"Translation: {translation}")

print("\n" + "="*100)

Using device: mps
Loading base model...
Loading LoRA adapter...
Loading tokenizer...
✅ Model loaded successfully!

Loaded 10 test samples
✅ Translation function ready
SANSKRIT TO ENGLISH TRANSLATION TESTS

────────────────────────────────────────────────────────────────────────────────────────────────────
Test 1/10: Mahabharata - Bhagavad Gita 1.1
────────────────────────────────────────────────────────────────────────────────────────────────────

📜 Sanskrit:
   धृतराष्ट्र उवाच। धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सवः। मामकाः पाण्डवाश्चैव किमकुर्वत सञ्जय॥

🎯 Expected Context:
   Dhritarashtra's opening question about the battle

🔄 Translating...

✨ Model Translation:
   The Dhr̄taśṭrā are the two parts that are situated in the earth. They are the two parts that are situated in the eastern and the western seas; they are also the two parts that are situated in the two continents.


────────────────────────────────────────────────────────────────────────────────────────────────────
Tes

In [28]:
from transformers import AutoTokenizer


MODEL_NAME = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)  # replace with your model
sample_text = "रामायण"
print(tokenizer.tokenize(sample_text))


['à¤°', 'à¤¾à¤', '®', 'à¤¾à¤', '¯', 'à¤£']
